In [1]:
import os
import io
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

import datetime

#dask
import dask.dataframe as dd

import warnings
warnings.filterwarnings("ignore")

### Path

In [2]:
path = r"\\VNCRP1002\\ud03$\\X266754\\My Documents\\Lapse\\Data\\"

### PFMC

In [3]:
filenames = glob.glob(path + "\*pfmc*.csv")
print(filenames)

['\\\\VNCRP1002\\\\ud03$\\\\X266754\\\\My Documents\\\\Lapse\\\\Data\\vul_pfmc_inf_monthly_2022_4.csv', '\\\\VNCRP1002\\\\ud03$\\\\X266754\\\\My Documents\\\\Lapse\\\\Data\\vul_pfmc_inf_monthly_2022_5.csv', '\\\\VNCRP1002\\\\ud03$\\\\X266754\\\\My Documents\\\\Lapse\\\\Data\\vul_pfmc_inf_monthly_2022_6.csv', '\\\\VNCRP1002\\\\ud03$\\\\X266754\\\\My Documents\\\\Lapse\\\\Data\\vul_pfmc_inf_monthly_2022_7.csv', '\\\\VNCRP1002\\\\ud03$\\\\X266754\\\\My Documents\\\\Lapse\\\\Data\\vul_pfmc_inf_monthly_2022_8.csv', '\\\\VNCRP1002\\\\ud03$\\\\X266754\\\\My Documents\\\\Lapse\\\\Data\\vul_pfmc_inf_monthly_2022_9.csv']


In [4]:
#list of all df
dfs = []
for f in filenames: 
    print(f)
    d = pd.read_csv(f, encoding="ISO-8859-1") 
    dfs.append(d) 
pfmc = pd.concat(dfs, ignore_index=True)
del dfs

\\VNCRP1002\\ud03$\\X266754\\My Documents\\Lapse\\Data\vul_pfmc_inf_monthly_2022_4.csv
\\VNCRP1002\\ud03$\\X266754\\My Documents\\Lapse\\Data\vul_pfmc_inf_monthly_2022_5.csv
\\VNCRP1002\\ud03$\\X266754\\My Documents\\Lapse\\Data\vul_pfmc_inf_monthly_2022_6.csv
\\VNCRP1002\\ud03$\\X266754\\My Documents\\Lapse\\Data\vul_pfmc_inf_monthly_2022_7.csv
\\VNCRP1002\\ud03$\\X266754\\My Documents\\Lapse\\Data\vul_pfmc_inf_monthly_2022_8.csv
\\VNCRP1002\\ud03$\\X266754\\My Documents\\Lapse\\Data\vul_pfmc_inf_monthly_2022_9.csv


In [5]:
# remove duplicates
pfmc = pfmc.drop_duplicates()

In [6]:
# create current and previous months for pfmc
# pfmc cur_month,  pfmc_prev_month
pfmc['val_dt']= pfmc['val_dt'].astype('datetime64[ns]')
pfmc['cur_month'] = pfmc['val_dt'].astype('datetime64[ns]').map(lambda x: x.strftime('%Y-%m'))
pfmc['prev_month'] =  ((pfmc['val_dt'] - pd.DateOffset(months=1)).dt.to_period("M")).astype(str)

In [7]:
pd.set_option('display.max_columns', None)
#pfmc.head()
pfmc[pfmc['agmt_pkge_id']=='V30059210000'][['agmt_pkge_id','prev_month', 'cur_month']]

,agmt_pkge_id,prev_month,cur_month
49041,V30059210000,2022-03,2022-04
1870004,V30059210000,2022-05,2022-06
3167383,V30059210000,2022-06,2022-07
3479468,V30059210000,2022-07,2022-08
4876885,V30059210000,2022-08,2022-09


### Events_Monthly

In [8]:
filenames = glob.glob(path + "\*events_monthly*.csv")
print(filenames)

['\\\\VNCRP1002\\\\ud03$\\\\X266754\\\\My Documents\\\\Lapse\\\\Data\\events_monthly_2022_4.csv', '\\\\VNCRP1002\\\\ud03$\\\\X266754\\\\My Documents\\\\Lapse\\\\Data\\events_monthly_2022_5.csv', '\\\\VNCRP1002\\\\ud03$\\\\X266754\\\\My Documents\\\\Lapse\\\\Data\\events_monthly_2022_6.csv', '\\\\VNCRP1002\\\\ud03$\\\\X266754\\\\My Documents\\\\Lapse\\\\Data\\events_monthly_2022_7.csv', '\\\\VNCRP1002\\\\ud03$\\\\X266754\\\\My Documents\\\\Lapse\\\\Data\\events_monthly_2022_8.csv', '\\\\VNCRP1002\\\\ud03$\\\\X266754\\\\My Documents\\\\Lapse\\\\Data\\events_monthly_2022_9.csv']


In [9]:
#list of all df
dfs = []
for f in filenames: 
    print(f)
    d = pd.read_csv(f, encoding="ISO-8859-1") 
    dfs.append(d) 
events = pd.concat(dfs, ignore_index=True)
del dfs

\\VNCRP1002\\ud03$\\X266754\\My Documents\\Lapse\\Data\events_monthly_2022_4.csv
\\VNCRP1002\\ud03$\\X266754\\My Documents\\Lapse\\Data\events_monthly_2022_5.csv
\\VNCRP1002\\ud03$\\X266754\\My Documents\\Lapse\\Data\events_monthly_2022_6.csv
\\VNCRP1002\\ud03$\\X266754\\My Documents\\Lapse\\Data\events_monthly_2022_7.csv
\\VNCRP1002\\ud03$\\X266754\\My Documents\\Lapse\\Data\events_monthly_2022_8.csv
\\VNCRP1002\\ud03$\\X266754\\My Documents\\Lapse\\Data\events_monthly_2022_9.csv


In [10]:
# remove duplicates
events = events.drop_duplicates()

In [11]:
# create current and previous months for events 
events['prcs_dt']= events['prcs_dt'].astype('datetime64[ns]')
events['evnt_cur_month'] = events['prcs_dt'].map(lambda x: x.strftime('%Y-%m'))
events['evnt_prev_month'] =  ((events['prcs_dt'] - pd.DateOffset(months=1)).dt.to_period("M")).astype(str)

In [14]:
# example
pd.set_option('display.max_columns', None)
events[events['agmt_pkge_id']=='V30059210000'][['agmt_pkge_id','evnt_desc','evnt_prev_month', 'evnt_cur_month']]

,agmt_pkge_id,evnt_desc,evnt_prev_month,evnt_cur_month
30556,V30059210000,Lapse With No Value,2022-04,2022-05


### Join PFMC Inf and Events to get Lapse policiies

In [16]:
# Merge pfmv and events
df = pfmc.merge(events[['agmt_pkge_id','evnt_desc','evnt_prev_month', 'evnt_cur_month']], 
              how='left', 
              left_on=['agmt_pkge_id', 'cur_month'], 
              right_on=['agmt_pkge_id', 'evnt_prev_month'], 
              suffixes=('', '_remove'))
df.drop([i for i in df.columns if 'remove' in i],axis=1, inplace=True)

In [17]:
df.shape

(4982168, 239)

In [32]:
# check
df[df['agmt_pkge_id']=='V30059210000'][['agmt_pkge_id','evnt_desc',
                                        'prev_month', 'cur_month',
                                        'evnt_prev_month', 'evnt_cur_month']]

,agmt_pkge_id,evnt_desc,prev_month,cur_month,evnt_prev_month,evnt_cur_month
49041,V30059210000,Lapse With No Value,2022-03,2022-04,2022-04,2022-05
1870004,V30059210000,Lapse With No Value,2022-05,2022-06,NaN,NaN
3167383,V30059210000,Lapse With No Value,2022-06,2022-07,NaN,NaN
3479468,V30059210000,Lapse With No Value,2022-07,2022-08,NaN,NaN
4876885,V30059210000,Lapse With No Value,2022-08,2022-09,NaN,NaN


In [18]:
#del pfmc, events

### Lapse and Surrender Indicators (reinstatement, to be done)

In [19]:
# forward fill indicators
df['evnt_desc']= df.groupby('agmt_pkge_id')['evnt_desc'].ffill()

In [20]:
# indicator
df['lapse_ind'] = (np.where(df['evnt_desc'].str.contains('Lapse', na=False),1, 0)).astype(int)
df['surrender_ind'] = (np.where(df['evnt_desc'].str.contains('Surrender',na=False),1, 0)).astype(int)

In [22]:
## create lag indicators
df['lapse_ind']= df['lapse_ind'].fillna(0)
df['lapse_ind_lag1'] = df.groupby('agmt_pkge_id')['lapse_ind'].shift(1)
df['lapse_ind_lag1']=df['lapse_ind_lag1'].fillna(0)

df['surrender_ind']= df['surrender_ind'].fillna(0)
df['surrender_ind_lag1'] = df.groupby('agmt_pkge_id')['surrender_ind'].shift(1)
df['surrender_ind_lag1']=df['surrender_ind_lag1'].fillna(0)

In [23]:
df['cur_status']= np.where(df['lapse_ind']==1, 'lapse', 
                           (np.where(df['surrender_ind']==1, 'surrender', "current")))

In [24]:
df['prior_status']= np.where(df['lapse_ind_lag1']==1, 'lapse', 
                           (np.where(df['surrender_ind_lag1']==1, 'surrender', "current")))

In [34]:
# check
df[df['agmt_pkge_id']=='965390970000'][['agmt_pkge_id','evnt_desc','lapse_ind', 'surrender_ind',
                                        'prior_status', 'cur_status',
                                        'prev_month', 'cur_month',
                                        'evnt_prev_month', 'evnt_cur_month']]

,agmt_pkge_id,evnt_desc,lapse_ind,surrender_ind,prior_status,cur_status,prev_month,cur_month,evnt_prev_month,evnt_cur_month
684,965390970000,Surrender,0,1,current,surrender,2022-03,2022-04,2022-04,2022-05


In [26]:
dgroup = df.groupby(["cur_month", "lapse_ind"], as_index=True)['lapse_ind'].count()
dgroup

cur_month  lapse_ind
2022-04    0            829299
           1               238
2022-05    0            829676
           1               245
2022-06    0            829721
           1               256
2022-07    0            830342
           1               312
2022-08    0            830213
           1               299
2022-09    0            831480
           1                87
Name: lapse_ind, dtype: int64

In [27]:
dgroup = df.groupby(["cur_month", "surrender_ind"], as_index=True)['agmt_pkge_id'].count()
dgroup

cur_month  surrender_ind
2022-04    0                828099
           1                  1438
2022-05    0                828521
           1                  1400
2022-06    0                828620
           1                  1357
2022-07    0                829055
           1                  1599
2022-08    0                829042
           1                  1470
2022-09    0                831558
           1                     9
Name: agmt_pkge_id, dtype: int64

### Transition Matrix

In [28]:
sample = df[df['cur_month']=='2022-08']
tmatrix = pd.crosstab(index=sample['prior_status'], columns=[sample['cur_status']]).reset_index()
tmatrix

cur_status,prior_status,current,lapse,surrender
0,current,828743,234,1464
1,lapse,0,65,0
2,surrender,0,0,6


In [29]:
pd.crosstab(index=sample['prior_status'], columns=[sample['cur_status']],normalize='index').round(4)*100

cur_status,current,lapse,surrender
prior_status,,,
current,99.8,0.03,0.18
lapse,0.0,100.00,0.00
surrender,0.0,0.00,100.00


#### check

In [1]:
df[(df['prior_status']=='current')&(df['cur_status']=='surrender')].head()

NameError: name 'df' is not defined

#### export to parquet

In [35]:
df=df[df['cur_month']<='2022-08']
df.to_parquet('pfmc_events_apr_aug.parquet')